In [1]:
pip install captcha

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 1.9 MB/s eta 0:00:00


In [2]:
import os
import cv2
import numpy as np
import random
import string
from captcha.image import ImageCaptcha
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, InputLayer
from tensorflow.keras.utils import to_categorical

In [3]:
#generate a random string of uppercase letters and digits with a specified length
def random_text(length=5):
    letters=string.ascii_uppercase + string.digits
    return ''.join(random.choice(letters) for i in range(length))

In [5]:
def generate_captcha_images(num_images=1000,output_dir='captchas'):
    os.makedirs(output_dir,exist_ok=True)#make sure that capthcas directory exists
    image = ImageCaptcha()#create an instance from captcha lib
    for i in range(num_images):
        text=random_text()#variable for random text funct
        image.write(text,os.path.join(output_dir, f'{text}.png'))#save the image as the name of captcha in png
generate_captcha_images()

In [26]:
'''def preprocess_image(image_path, img_width=100, img_height=40):
    img=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(img_width, img_height))
    img=img/255.0#normalisation to [0, 1]
    img=np.expand_dims(img, axis=-1)#adds another column to make it 3d at the last
    return img'''
def preprocess_image(image_path, img_width=100, img_height=40):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    img = cv2.resize(img, (img_width, img_height))
    img = img / 255.0  # Normalize to [0, 1]
    img = np.expand_dims(img, axis=-1)
    return img


In [27]:
def load_data(data_dir,img_width=100,img_height=40):
    X,y=[],[]
    for filename in os.listdir(data_dir):
        if filename.endswith('.png'):
            image_path=os.path.join(data_dir,filename)#updating image path
            X.append(preprocess_image(image_path,img_width,img_height))
            y.append(filename.split('.')[0])
    return np.array(X), np.array(y)


In [28]:
X,y=load_data('captchas')
captcha_length=5
num_classes=36
char_list=string.ascii_uppercase+string.digits
char_to_index={char: idx for idx, char in enumerate(char_list)}#dictionary that maps each character in char_list to a unique integer index

In [29]:
def encode_labels(labels,captcha_length,num_classes):
    encoded=np.zeros((len(labels),captcha_length,num_classes),dtype=np.uint8)
    for i, label in enumerate(labels):
        for j, char in enumerate(label):
            encoded[i,j,char_to_index[char]]=1
    return encoded

In [30]:
y_encoded=encode_labels(y,captcha_length,num_classes)

In [31]:
def create_model(input_shape, captcha_length,num_classes):
    model=Sequential()
    model.add(InputLayer(input_shape=input_shape))
    model.add(Conv2D(32,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64, (3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dense(captcha_length*num_classes,activation='softmax'))
    model.add(Reshape((captcha_length,num_classes)))
    return model
input_shape=(40,100,1)
model=create_model(input_shape,captcha_length,num_classes)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X,y_encoded,epochs=30,batch_size=32,validation_split=0.2)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 191ms/step - accuracy: 0.0277 - loss: 3.5938 - val_accuracy: 0.0300 - val_loss: 3.5841
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 148ms/step - accuracy: 0.0326 - loss: 3.5804 - val_accuracy: 0.0320 - val_loss: 3.5853
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 206ms/step - accuracy: 0.0359 - loss: 3.5760 - val_accuracy: 0.0390 - val_loss: 3.5843
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 160ms/step - accuracy: 0.0340 - loss: 3.5752 - val_accuracy: 0.0310 - val_loss: 3.5912
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 154ms/step - accuracy: 0.0385 - loss: 3.5682 - val_accuracy: 0.0360 - val_loss: 3.5878
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 164ms/step - accuracy: 0.0409 - loss: 3.5659 - val_accuracy: 0.0230 - val_loss: 3.5878
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 153ms/step - accuracy: 0.0357 - loss: 3.5646 - val_accuracy: 0.0250 - val_loss: 3.5858
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 149ms/step - accuracy: 0.0367 - loss: 3.5694 - val_accuracy: 0.

In [32]:
def decode_captcha(model,image_path,char_list,char_to_index):
    img=preprocess_image(image_path)
    img=np.expand_dims(img, axis=0)
    prediction=model.predict(img)
    decoded_text=''.join([char_list[np.argmax(char)] for char in prediction[0]])
    return decoded_text

In [33]:
index_to_char={idx: char for char,idx in char_to_index.items()}


In [36]:
captcha_image_path='/content/captchas/19Q0X.png'
decoded_text=decode_captcha(model,captcha_image_path,index_to_char, char_to_index)
print(f'Decoded CAPTCHA text: {decoded_text}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Decoded CAPTCHA text: A54MF
